In [17]:
!pip install -U accelerate 

  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rllm 0.0.1 requires ray==2.23, but you have ray 2.37.0 which is incompatible.


In [18]:
from datasets import load_dataset
from adapters import AutoAdapterModel

from transformers import GPT2Config
from transformers import GPT2Tokenizer
import wandb

In [19]:
wandb.login(key="d1209ac0e4a568567e407d3afef99099ea87a4b1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [20]:
%env WANDB_PROJECT=peft
%env WANDB_ENTITY=SC4001

env: WANDB_PROJECT=peft
env: WANDB_ENTITY=SC4001


In [21]:
dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

Path, databilder:  rotten_tomatoes


{'train': 8530, 'validation': 1066, 'test': 1066}

In [22]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [23]:
from adapters import IA3Config, SeqBnConfig, LoRAConfig, ReftConfig, CompacterConfig

ia3_config = IA3Config()
seq_bn_config = SeqBnConfig()
lora_config = LoRAConfig()
compacter_config = CompacterConfig()
reft_config = ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=True)

In [33]:
config = GPT2Config.from_pretrained(
    "gpt2",
    num_labels=5,
)
model = AutoAdapterModel.from_pretrained(
    "gpt2",
    config=config,
)

In [34]:
runname = "gpt-rotten_tomatoes-lora"
model.add_adapter("rotten_tomatoes", config=lora_config)
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
)

# Activate the adapter
model.train_adapter("rotten_tomatoes")
# model.set_active_adapters("rotten_tomatoes")

# calculate the number of training parameters
# total = 0
# for p in model.parameters():
#     if p.requires_grad:
#         total += p.numel()
# print(f"Total number of trainable parameters: {total}")

In [35]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    # per_device_train_batch_size=128,
    # per_device_eval_batch_size=128,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    run_name=runname
)

def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [36]:
trainer.train()

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [8, 12, 80, 80]] is at version 12; expected version 11 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
trainer.evaluate(dataset["validation"])